https://github.com/Motamensalih/RAG_First_Project/blob/main/examples/1-managing-data/embeddings.ipynb

In [6]:
import pandas as pd
df = pd.read_csv('/content/RAG_First_Project/top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [7]:
#!pip install huggingface-hub==0.25.2 # Install huggingface-hub version 0.25.2 which has the function.
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [ ]:
!pip install huggingface-hub==0.25.2 # Install huggingface-hub version 0.25.2 which has the function.

In [8]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [9]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [10]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-10-36d8e74a557e>:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [11]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [12]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [13]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Bruno Rocca Barbaresco Rabaja 2000', 'region': 'Barbaresco, Piedmont, Italy', 'variety': 'Red Wine', 'rating': 97.0, 'notes': 'Unbelievable aromas, with rich, ripe plum but also mineral, tobacco and cedar undertones. Full-bodied, with lovely ripe tannins and a unctuous combination of ripe fruit and light toasty oak. Goes on and on. Fabulous. Greatest wine ever from Bruno Rocca. Best after 2007. 1,500 cases made. (JS)'} score: 0.6026552019512349
{'name': 'Jorge Ordonez Number 4 Esencia (375ML half-bottle) 2004', 'region': 'Spain', 'variety': 'Boutique', 'rating': 99.0, 'notes': 'Esencia is a unique wine that incorporates the raisined muscat grape.  After 24 months in barrel, we achieve a partial fermentation of the must.  Alois Kracher, through this wine, sought to convey the essence of the village of Almáchar, in the heart of the Axarquía, famous from time immemorial for its delicious muscat grapes and raisins.  '} score: 0.5889358363836907
{'name': "Brovia Ca'Mia Barolo (1.5

In [14]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [3]:
!pip install openai==1.11.1

  Using cached openai-1.11.1-py3-none-any.whl.metadata (18 kB)
Using cached openai-1.11.1-py3-none-any.whl (226 kB)
  Attempting uninstall: openai
    Found existing installation: openai 0.27.8
    Uninstalling openai-0.27.8:
      Successfully uninstalled openai-0.27.8


In [1]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

TypeError: Client.__init__() got an unexpected keyword argument 'proxies'

In [ ]:
!git clone https://github.com/Motamensalih/RAG_First_Project.git

In [ ]:
%cd RAG_First_Project
!pip install -r requirements.txt

In [2]:
import pandas as pd
df = pd.read_csv('/content/RAG_First_Project/top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records')
df

,name,region,variety,rating,notes
0,3 Rings Reserve Shiraz 2004,"Barossa Valley, Barossa, South Australia, Aust...",Red Wine,96.0,Vintage Comments : Classic Barossa vintage con...
1,Abreu Vineyards Cappella 2007,"Napa Valley, California",Red Wine,96.0,Cappella is a proprietary blend of two clones ...
2,Abreu Vineyards Cappella 2010,"Napa Valley, California",Red Wine,98.0,Cappella is one of the oldest vineyard sites i...
3,Abreu Vineyards Howell Mountain 2008,"Howell Mountain, Napa Valley, California",Red Wine,96.0,When David purchased this Howell Mountain prop...
4,Abreu Vineyards Howell Mountain 2009,"Howell Mountain, Napa Valley, California",Red Wine,98.0,"As a set of wines, it is hard to surpass the f..."
...,...,...,...,...,...
1360,Lewis Cellars Alec's Blend Red 2002,"Napa Valley, California",Red Wine,96.0,Number 12 on
1361,Lewis Cellars Cabernet Sauvignon 2002,"Napa Valley, California",Red Wine,96.0,Showcasing the unique personalities of small h...
1362,Lewis Cellars Cuvee L Cabernet Sauvignon 2015,"Napa Valley, California",Red Wine,96.0,"Straight from James Fenimore Cooper’s novel, L..."
1363,Lewis Cellars Reserve Cabernet Sauvignon 2010,"Napa Valley, California",Red Wine,96.0,


In [3]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [4]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

In [5]:
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [6]:
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

<ipython-input-6-eefb947fdd31>:1: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [7]:
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [8]:
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Kapcsandy Family Winery State Lane Cabernet Sauvignon Grand Vin 2017', 'region': 'Yountville, Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': '100% Cabernet Sauvignon'} score: 0.7492028166137619
{'name': 'Lewis Cellars Cabernet Sauvignon 2002', 'region': 'Napa Valley, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Showcasing the unique personalities of small hillside vineyards from Pritchard Hill, Oakville and Rutherford, the 2002 Napa Valley Cabernet delivers compelling aromas of mocha, ripe berries, tobacco and sweet oak spice. The wine is 100% Cabernet Sauvignon, complex, rich and focused. With a deep core of black fruit and traces of briar and vanilla, it turns chocolaty and long on the palate with serious, integrated tannins.'} score: 0.7331372908744138
{'name': 'Anakota Helena Montana Vineyard Cabernet Sauvignon 2013', 'region': 'Knights Valley, Sonoma County, California', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Blend: 1

In [9]:
import csv

# Data to be written to the CSV file
data = [
    {"Inventor": "Thomas Edison", "Invention": "Light Bulb"},
    {"Inventor": "Alexander Graham Bell", "Invention": "Telephone"},
    {"Inventor": "Nikola Tesla", "Invention": "Alternating Current"},
    {"Inventor": "Galileo Galilei", "Invention": "Telescope"},
    {"Inventor": "Louis Pasteur", "Invention": "Vaccination"},
    {"Inventor": "Guglielmo Marconi", "Invention": "Radio"},
    {"Inventor": "Wright Brothers", "Invention": "Airplane"},
    {"Inventor": "Tim Berners-Lee", "Invention": "World Wide Web"},
    {"Inventor": "Steve Jobs", "Invention": "iPhone"},
    {"Inventor": "Bill Gates", "Invention": "Microsoft Windows"}
]

# Name of the CSV file to be created
filename = "inventors_and_inventions.csv"

# Writing data to the CSV file
with open(filename, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["Inventor", "Invention"])

    # Write the header (column names)
    writer.writeheader()

    # Write the data rows
    for row in data:
        writer.writerow(row)

print(f"File {filename} has been created successfully!")

File inventors_and_inventions.csv has been created successfully!


In [ ]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)
hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode("99 points Cabernet Sauvignon from Napa Valley").tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

نعم، في الكود الذي كتبته:

```python
vector=encoder.encode(doc["Invention"]).tolist()
```

`"Invention"` هو اسم العمود في ملف CSV الذي يحتوي على نص الاختراع. هذا يعني أن الكود يتوقع أن يكون لديك عمود في ملف CSV باسم `Invention`، وسيتم استخدام النص الموجود في هذا العمود لإنشاء التضمينات (embeddings) باستخدام النموذج `SentenceTransformer`.

### إذا كان اسم العمود مختلفًا:
إذا كان اسم العمود في ملف CSV الخاص بك مختلفًا (على سبيل المثال، `اختراع` أو `invention_text`)، فستحتاج إلى تعديل الكود ليعكس الاسم الصحيح للعمود. على سبيل المثال:

```python
vector=encoder.encode(doc["اختراع"]).tolist()  # إذا كان اسم العمود "اختراع"
```

أو:

```python
vector=encoder.encode(doc["invention_text"]).tolist()  # إذا كان اسم العمود "invention_text"
```

### مثال لملف CSV مع أعمدة مختلفة:
إذا كان ملف CSV الخاص بك يحتوي على أعمدة مثل `اسم المخترع` و`اختراع`، فسيبدو كالتالي:

```
اسم المخترع,اختراع
توماس إديسون,المصباح الكهربائي
ألكسندر جراهام بيل,الهاتف
نيكولا تيسلا,التيار المتردد
```

في هذه الحالة، ستحتاج إلى تعديل الكود ليكون:

```python
vector=encoder.encode(doc["اختراع"]).tolist()  # استخدام العمود "اختراع"
```

### الكود المعدل مع أسماء أعمدة مختلفة:
إذا كانت أسماء الأعمدة في ملف CSV هي `اسم المخترع` و`اختراع`، فإليك الكود المعدل:

```python
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import csv

# تهيئة نموذج التشفير
encoder = SentenceTransformer('all-MiniLM-L6-v2')

# إنشاء مثيل Qdrant في الذاكرة
qdrant = QdrantClient(":memory:")

# إعادة إنشاء المجموعة (collection) للمخترعين
qdrant.recreate_collection(
    collection_name="inventors_and_inventions",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),
        distance=models.Distance.COSINE
    )
)

# مسار ملف CSV
data_path = "inventors.csv"  # تأكد من أن الملف موجود في نفس الدليل أو قم بتحديد المسار الكامل

# قراءة البيانات من ملف CSV
data = []
with open(data_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)  # قراءة الملف كقاموس
    for row in reader:
        data.append(row)

# تحميل النقاط (المخترعين واختراعاتهم) إلى مجموعة Qdrant
qdrant.upload_points(
    collection_name="inventors_and_inventions",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["اختراع"]).tolist(),  # استخدام العمود "اختراع"
            payload=doc
        ) for idx, doc in enumerate(data)
    ]
)

# البحث عن اختراعات مشابهة بناءً على استعلام
query = "جهاز ينتج الضوء"  # مثال لاستعلام
hits = qdrant.search(
    collection_name="inventors_and_inventions",
    query_vector=encoder.encode(query).tolist(),  # تشفير الاستعلام
    limit=3  # تحديد عدد النتائج
)

# طباعة نتائج البحث
for hit in hits:
    print(hit.payload, "score:", hit.score)
```

### ملاحظة:
- تأكد من أن أسماء الأعمدة في الكود تتطابق مع أسماء الأعمدة في ملف CSV.
- إذا كان ملف CSV يحتوي على رأس (header)، فسيتم استخدامه تلقائيًا عند قراءة الملف باستخدام `csv.DictReader`.

شغال

In [12]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import csv

# تهيئة نموذج التشفير
encoder = SentenceTransformer('all-MiniLM-L6-v2')  # نموذج لإنشاء التضمينات

# إنشاء مثيل Qdrant في الذاكرة
qdrant = QdrantClient(":memory:")

# إعادة إنشاء المجموعة (collection) للمخترعين
qdrant.recreate_collection(
    collection_name="inventors_and_inventions",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # حجم المتجهات يعتمد على النموذج
        distance=models.Distance.COSINE
    )
)

# مسار ملف CSV الذي يحتوي على البيانات
data_path = "/content/inventors_and_inventions.csv"  # تأكد من أن الملف موجود في نفس الدليل أو قم بتحديد المسار الكامل

# قراءة البيانات من ملف CSV
data = []
with open(data_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)  # قراءة الملف كقاموس
    for row in reader:
        data.append(row)

# تحميل النقاط (المخترعين واختراعاتهم) إلى مجموعة Qdrant
qdrant.upload_points(
    collection_name="inventors_and_inventions",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Invention"]).tolist(),  # تشفير نص الاختراع
            payload=doc
        ) for idx, doc in enumerate(data)  # البيانات مخزنة في المتغير data
    ]
)

# البحث عن اختراعات مشابهة بناءً على استعلام
query = "A device that produces light"  # مثال لاستعلام
hits = qdrant.search(
    collection_name="inventors_and_inventions",
    query_vector=encoder.encode(query).tolist(),  # تشفير الاستعلام
    limit=1  # تحديد عدد النتائج
)

# طباعة نتائج البحث
for hit in hits:
    print(hit.payload, "score:", hit.score)

<ipython-input-12-649dc56ff0de>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


{'Inventor': 'Thomas Edison', 'Invention': 'Light Bulb'} score: 0.632008530571912


In [ ]:
data_path = "/content/inventors_and_inventions.csv"

collection_name="inventors_and_inventions",



In [10]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

# Initialize the encoder model
encoder = SentenceTransformer('all-MiniLM-L6-v2')  # Model to create embeddings

# Create an in-memory Qdrant instance
qdrant = QdrantClient(":memory:")

# Recreate the collection for inventors
qdrant.recreate_collection(
    collection_name="inventors_and_inventions",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # Vector size is defined by the model
        distance=models.Distance.COSINE
    )
)

# Data for inventors and their inventions
data = [
    {"Inventor": "Thomas Edison", "Invention": "Light Bulb"},
    {"Inventor": "Alexander Graham Bell", "Invention": "Telephone"},
    {"Inventor": "Nikola Tesla", "Invention": "Alternating Current"},
    {"Inventor": "Galileo Galilei", "Invention": "Telescope"},
    {"Inventor": "Louis Pasteur", "Invention": "Vaccination"},
    {"Inventor": "Guglielmo Marconi", "Invention": "Radio"},
    {"Inventor": "Wright Brothers", "Invention": "Airplane"},
    {"Inventor": "Tim Berners-Lee", "Invention": "World Wide Web"},
    {"Inventor": "Steve Jobs", "Invention": "iPhone"},
    {"Inventor": "Bill Gates", "Invention": "Microsoft Windows"}
]

# Upload points (inventors and inventions) to the Qdrant collection
qdrant.upload_points(
    collection_name="inventors_and_inventions",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["Invention"]).tolist(),  # Encode the invention text
            payload=doc
        ) for idx, doc in enumerate(data)  # data is the variable holding all the inventors
    ]
)

# Search for similar inventions based on a query
query = "A device that produces light"  # Example query
hits = qdrant.search(
    collection_name="inventors_and_inventions",
    query_vector=encoder.encode(query).tolist(),  # Encode the query
    limit=3  # Limit the number of results
)

# Print the search results
for hit in hits:
    print(hit.payload, "score:", hit.score)

<ipython-input-10-54e9162d9f8f>:11: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


{'Inventor': 'Thomas Edison', 'Invention': 'Light Bulb'} score: 0.632008530571912
{'Inventor': 'Galileo Galilei', 'Invention': 'Telescope'} score: 0.37578558618623886
{'Inventor': 'Steve Jobs', 'Invention': 'iPhone'} score: 0.2843923830227583


In [13]:
!wget https://huggingface.co/datasets/valentij/test.csv/resolve/main/nn_test.csv

--2025-01-22 23:04:32--  https://huggingface.co/datasets/valentij/test.csv/resolve/main/nn_test.csv
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.11, 3.165.160.61, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.hf.co/repos/c5/d2/c5d2041977986f42917562e9e6d8514be8b3bf15869688ea5ebc672f95bdbbee/91e43ece29fa39ad2e625cb4b31c123e5cd690f99bfe03c3546ea59fe20e393f?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27nn_test.csv%3B+filename%3D%22nn_test.csv%22%3B&response-content-type=text%2Fcsv&Expires=1737590672&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczNzU5MDY3Mn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5oZi5jby9yZXBvcy9jNS9kMi9jNWQyMDQxOTc3OTg2ZjQyOTE3NTYyZTllNmQ4NTE0YmU4YjNiZjE1ODY5Njg4ZWE1ZWJjNjcyZjk1YmRiYmVlLzkxZTQzZWNlMjlmYTM5YWQyZTYyNWNiNGIzMWMxMjNlNWNkNjkwZjk5YmZlMDNjMzU0NmVhNTlmZTIwZTM5M2Y%7EcmVzc

https://huggingface.co/datasets/valentij/test.csv/viewer?row=3

ayhgشغال

In [1]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer
import csv

# تهيئة نموذج التشفير
encoder = SentenceTransformer('all-MiniLM-L6-v2')  # نموذج لإنشاء التضمينات

# إنشاء مثيل Qdrant في الذاكرة
qdrant = QdrantClient(":memory:")

# إعادة إنشاء المجموعة (collection) للمخترعين
qdrant.recreate_collection(
    collection_name="nn_test",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(),  # حجم المتجهات يعتمد على النموذج
        distance=models.Distance.COSINE
    )
)

# مسار ملف CSV الذي يحتوي على البيانات
data_path = "/content/nn_test.csv"  # تأكد من أن الملف موجود في نفس الدليل أو قم بتحديد المسار الكامل

# قراءة البيانات من ملف CSV
data = []
with open(data_path, mode='r', encoding='utf-8') as file:
    reader = csv.DictReader(file)  # قراءة الملف كقاموس
    for row in reader:
        data.append(row)

# تحميل النقاط (المخترعين واختراعاتهم) إلى مجموعة Qdrant
qdrant.upload_points(
    collection_name="nn_test",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["text"]).tolist(),  # تشفير نص الاختراع
            payload=doc
        ) for idx, doc in enumerate(data)  # البيانات مخزنة في المتغير data
    ]
)

# البحث عن اختراعات مشابهة بناءً على استعلام
query = "What is the medication mentioned in the data that helped treat depression, insomnia and anxiety?"  # مثال لاستعلام
hits = qdrant.search(
    collection_name="nn_test",
    query_vector=encoder.encode(query).tolist(),  # تشفير الاستعلام
    limit=1  # تحديد عدد النتائج
)

# طباعة نتائج البحث
for hit in hits:
    print(hit.payload, "score:", hit.score)

<ipython-input-1-96cb74f0b4d6>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


{'text': '"Excellent medicine for anxiety, depression and insomnia."', 'label': '1'} score: 0.8083904173713107


<ipython-input-1-96cb74f0b4d6>:12: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(
{'text': '"Excellent medicine for anxiety, depression and insomnia."', 'label': '1'} score: 0.8083904173713107

In [2]:
import os
import torch

from pymilvus import connections, Collection, CollectionSchema, DataType, FieldSchema, utility
from PyPDF2 import PdfReader
from transformers import DPRContextEncoder, DPRQuestionEncoderTokenizer

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

ModuleNotFoundError: No module named 'pymilvus'

In [4]:
!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.7 MB/s eta 0:00:00


In [3]:
import os
import torch

from qdrant_client import QdrantClient, models  # استبدال PyMilvus بـ Qdrant
from PyPDF2 import PdfReader
from transformers import DPRContextEncoder, DPRQuestionEncoderTokenizer

# تهيئة Qdrant
qdrant_client = QdrantClient(host="localhost", port=6333)

# ... بقية التعليمات البرمجية ...

ModuleNotFoundError: No module named 'PyPDF2'

In [5]:
!pip install faiss-cpu==1.7.4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 28.3 MB/s eta 0:00:00


In [6]:
import faiss
import numpy as np

In [7]:
dimension = 128  # أبعاد متجهاتك
index = faiss.IndexFlatL2(dimension)

In [8]:
data = np.random.rand(1000, dimension).astype('float32')  # مثال لبيانات عشوائية
index.add(data)

In [9]:
query = np.random.rand(1, dimension).astype('float32')  # استعلام البحث
k = 5  # عدد النتائج المراد إرجاعها
D, I = index.search(query, k)  # D: المسافات، I: فهارس المتجهات

https://jboothomas.medium.com/rag-and-the-challenge-of-raw-to-embedded-dataset-size-4dbe70722cd2

In [11]:
import os
import torch
import faiss  # استيراد Faiss

from PyPDF2 import PdfReader
from transformers import DPRContextEncoder, DPRQuestionEncoderTokenizer

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# تحديد أبعاد المتجهات
dimension = 768  # افتراض أن أبعاد المتجهات هي 768

# إنشاء فهرس Faiss
index = faiss.IndexFlatL2(dimension)

# دالة لتحميل بيانات PDF وإنشاء تضمينات
def load_and_embed_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    # إنشاء تضمينات باستخدام DPRContextEncoder
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).pooler_output

    # تحويل التضمينات إلى تنسيق NumPy
    embeddings = embeddings.cpu().numpy()

    return embeddings

# مثال لتحميل ملف PDF وإضافة التضمينات إلى فهرس Faiss
pdf_path = "your_pdf_file.pdf"  # استبدل بمسار ملف PDF الخاص بك
embeddings = load_and_embed_pdf(pdf_path)
index.add(embeddings)

# مثال للبحث عن متجهات مشابهة
query_embeddings = load_and_embed_pdf("/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf")  # استبدل بمسار ملف PDF للاستعلام
k = 5  # عدد النتائج المراد إرجاعها
D, I = index.search(query_embeddings, k)  # D: المسافات، I: فهارس المتجهات

# طباعة نتائج البحث
print("فهارس النتائج:", I)
print("المسافات:", D)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


FileNotFoundError: [Errno 2] No such file or directory: 'your_pdf_file.pdf'

In [12]:
import os
import torch
import faiss  # استيراد Faiss

from PyPDF2 import PdfReader
from transformers import DPRContextEncoder, DPRQuestionEncoderTokenizer

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

# تحديد أبعاد المتجهات
dimension = 768  # افتراض أن أبعاد المتجهات هي 768

# إنشاء فهرس Faiss
index = faiss.IndexFlatL2(dimension)

# دالة لتحميل بيانات PDF وإنشاء تضمينات
def load_and_embed_pdf(pdf_path):
    reader = PdfReader(pdf_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()

    # إنشاء تضمينات باستخدام DPRContextEncoder
    inputs = tokenizer(text, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).pooler_output

    # تحويل التضمينات إلى تنسيق NumPy
    embeddings = embeddings.cpu().numpy()

    return embeddings

# مثال لتحميل ملف PDF وإضافة التضمينات إلى فهرس Faiss
pdf_path = "/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"  # استبدل بمسار ملف PDF الخاص بك
embeddings = load_and_embed_pdf(pdf_path)
index.add(embeddings)

# مثال للبحث عن متجهات مشابهة
query_embeddings = load_and_embed_pdf("/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf")  # استبدل بمسار ملف PDF للاستعلام
k = 5  # عدد النتائج المراد إرجاعها
D, I = index.search(query_embeddings, k)  # D: المسافات، I: فهارس المتجهات

# طباعة نتائج البحث
print("فهارس النتائج:", I)
print("المسافات:", D)

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


RuntimeError: The size of tensor a (2382) must match the size of tensor b (512) at non-singleton dimension 1

In [ ]:
def connect_to_milvus():
    connections.connect(alias="default", host="1.2.3.4", port="19530")
    print("Connected to Milvus.")

In [13]:
import os
import torch
import faiss  # استيراد Faiss

from PyPDF2 import PdfReader
from transformers import DPRContextEncoder, DPRQuestionEncoderTokenizer

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")

Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
import faiss

def connect_to_faiss(dimension=768):
    """
    Creates a Faiss index and returns it.

    Args:
        dimension: The dimensionality of the vectors to be indexed.

    Returns:
        A Faiss index.
    """
    index = faiss.IndexFlatL2(dimension)
    print("Faiss index created.")
    return index

In [15]:
index = connect_to_faiss()  # إنشاء فهرس Faiss

# ... (بقية الكود) ...

# إضافة بيانات إلى فهرس Faiss
index.add(embeddings)

# البحث في فهرس Faiss
D, I = index.search(query_embeddings, k)

# ... (بقية الكود) ...

Faiss index created.


NameError: name 'embeddings' is not defined

In [ ]:
RAG والتحدي المتمثل في حجم مجموعة البيانات الخام إلى المضمن.
جبوتوماس
جبوتوماس

·
اتبع

وقت القراءة: 8 دقائق
·
يوليو 22, 2024





مع زيادة عمليات نشر RAG المؤسسية ، من المثير للاهتمام دائما معرفة كيفية عمل الأشياء تحت الأغطية. مع وضع ذلك في الاعتبار في "كيفية التدوين" ، سأذهب إلى الكود لتضمين أجزاء نصية من الملفات في قاعدة بيانات متجهة والنظر في مساحة التخزين المطلوبة.

عند الاستفادة من قاعدة بيانات متجهة لحالة استخدام RAG، نبحث عبر عمليات التضمين استنادا إلى استعلام المستخدم ونرجع النتائج الأكثر صلة. ثم يتم تمرير النص المقترن بهذه النتائج والاستعلام الأولي إلى LLM حتى يتمكن من إنشاء الاستجابة بناء على هذا السياق المحسن.

يجب ألا تخزن قاعدة بيانات المتجهات تمثيل المتجه فحسب ، بل يجب أن تخزن أيضا النص الخام المرتبط ، لذلك دعنا نحصل على هذا الإعداد ثم نرى تأثير كمية كبيرة من بيانات المصدر على التخزين المطلوب ل vectorDB.

اعداد
لدي بالفعل مثيل milvus يعمل داخل مجموعة kubernetes باستخدام Pure Storage Flashblade للواجهة الخلفية S3. تم تثبيت Milvus باستخدام عامل التشغيل Milvus وتم تطبيق values.yaml التالية أثناء نشر المثيل:

apiVersion: milvus.io/v1beta1
kind: Milvus
metadata:
  name: jbt-milvus
  labels:
    app: milvus
spec:
  mode: cluster # Omit other fields ...
  config:
    minio:
      # your bucket name
      bucketName: jbt-milvus
      # Optional, config the prefix of the bucket milvus will use
      rootPath: milvus/jbt-milvus
      useSSL: false
  dependencies:
    storage:
      # enable external object storage
      external: true
      type: S3 # MinIO | S3
      # the endpoint of AWS S3
      endpoint: flashblade.purestorage.com
      #useSSL: false
      # the secret storing the access key and secret key
      secretRef: "jbt-milvus-s3-secret"
لسهولة الاستخدام ، أضفت أيضا خدمة موازنة التحميل للحصول على عنوان IP مرئي من metalLB من محطة العمل الخاصة بي ، وإليك تكوين k8s السريع لذلك:

apiVersion: v1
kind: Service
metadata:
  name: jbt-milvus-loadbalancer
spec:
  type: LoadBalancer
  selector:
    app.kubernetes.io/component: proxy
    app.kubernetes.io/instance: jbt-milvus
    app.kubernetes.io/name: milvus
    milvus.io/service: "true"
  ports:
    - protocol: TCP
      port: 19530
      targetPort: 19530
يحتوي "jbt-milvus-s3-secret" على الوصول إلى S3 ذي الصلة والمفاتيح السرية لحساب كائن S3 ، وإليك تخطيط yaml المطلوب:

apiVersion: v1
kind: Secret
metadata:
  name: jbt-milvus-s3-secret
type: Opaque
stringData:
  accesskey: PSFB*****BJEIA
  secretkey: A121*****eJOEN
يؤدي تطبيق yamls أعلاه إلى تشغيل مثيل milvus يمكن الوصول إليه من خارج مجموعة k8s ، ويجب أن يبدو مشابها لما يلي:

NAME                                             READY   STATUS    RESTARTS      AGE   IP               NODE       NOMINATED NODE   READINESS GATES
jbt-milvus-etcd-0                                1/1     Running   0             43h   172.168.96.110   jbt-54-k   <none>           <none>
jbt-milvus-etcd-1                                1/1     Running   0             45h   172.168.96.151   jbt-51-k   <none>           <none>
jbt-milvus-etcd-2                                1/1     Running   0             13d   172.168.96.163   jbt-51-k   <none>           <none>
jbt-milvus-milvus-datacoord-764f9475c5-7kjcb     1/1     Running   0             18h   172.168.96.226   jbt-52-k   <none>           <none>
jbt-milvus-milvus-datanode-7f899c8c58-7mj5f      1/1     Running   0             18h   172.168.96.241   jbt-52-k   <none>           <none>
jbt-milvus-milvus-indexcoord-5b5f8655f-b5qcw     1/1     Running   0             18h   172.168.96.109   jbt-54-k   <none>           <none>
jbt-milvus-milvus-indexnode-8d86c6db8-2c5mt      1/1     Running   0             18h   172.168.96.216   jbt-52-k   <none>           <none>
jbt-milvus-milvus-proxy-5dcccff4b-chngq          1/1     Running   0             18h   172.168.96.233   jbt-52-k   <none>           <none>
jbt-milvus-milvus-querycoord-fff7966b7-gm89l     1/1     Running   1 (38h ago)   38h   172.168.96.189   jbt-51-k   <none>           <none>
jbt-milvus-milvus-querynode-0-54f76d6f6f-8cw2g   1/1     Running   0             17h   172.168.96.113   jbt-54-k   <none>           <none>
jbt-milvus-milvus-rootcoord-99477cd9f-zm7cp      1/1     Running   0             18h   172.168.96.229   jbt-52-k   <none>           <none>
jbt-milvus-pulsar-bookie-0                       1/1     Running   0             28d   172.168.96.49    jbt-53-k   <none>           <none>
jbt-milvus-pulsar-bookie-1                       1/1     Running   0             28d   172.168.96.95    jbt-54-k   <none>           <none>
jbt-milvus-pulsar-bookie-2                       1/1     Running   0             45h   172.168.96.165   jbt-51-k   <none>           <none>
jbt-milvus-pulsar-broker-0                       1/1     Running   0             13d   172.168.96.142   jbt-51-k   <none>           <none>
jbt-milvus-pulsar-proxy-0                        1/1     Running   0             23h   172.168.96.230   jbt-52-k   <none>           <none>
jbt-milvus-pulsar-recovery-0                     1/1     Running   0             45h   172.168.96.172   jbt-51-k   <none>           <none>
jbt-milvus-pulsar-zookeeper-0                    1/1     Running   0             44h   172.168.96.156   jbt-51-k   <none>           <none>
jbt-milvus-pulsar-zookeeper-1                    1/1     Running   0             45h   172.168.96.160   jbt-51-k   <none>           <none>
jbt-milvus-pulsar-zookeeper-2                    1/1     Running   0             28d   172.168.96.111   jbt-54-k   <none>           <none>
داخل دفتر ملاحظات Jupyter ، أقوم بتحميل نواة مثبتة على المتطلبات التالية: pymilvus 2.4.3 و PyPDF2 3.0.1 و torch 2.3.0 و Transformers 4.41.2.

رمز
تتمثل الخطوة الأولى في تحميل الوحدات النمطية المطلوبة الخاصة بنا الرمز المميز والنموذج:

import os
import torch

from pymilvus import connections, Collection, CollectionSchema, DataType, FieldSchema, utility
from PyPDF2 import PdfReader
from transformers import DPRContextEncoder, DPRQuestionEncoderTokenizer

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
model = DPRContextEncoder.from_pretrained("facebook/dpr-ctx_encoder-single-nq-base")
ثم أقوم بتحديد الوظائف التالية ، الاتصال بقاعدة بيانات milvus الخاصة بنا:

def connect_to_milvus():
    connections.connect(alias="default", host="1.2.3.4", port="19530")
    print("Connected to Milvus.")
إنشاء مجموعة milvus ووظيفة لإدخال بياناتنا. هنا أقوم بتعريف المخطط باستخدام المعرف والتضمينات والنص لحقولي:

def create_collection(collection_name, dim=768):
    exists = utility.has_collection(collection_name)

    if not exists:
        print(f"Collection {collection_name} does not exist.")
        fields = [
            FieldSchema(name="id", dtype=DataType.INT64, is_primary=True, auto_id=True),
            FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=dim),
            FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512)
        ]
        schema = CollectionSchema(fields, description="Embedding Collection")
        collection = Collection(name=collection_name, schema=schema)
        print(f"Collection {collection_name} created.")
    else:
        collection = Collection(name=collection_name)
        print(f"Collection {collection_name} already exists.")
    return collection
def insert_embeddings(collection, embeddings, texts):
    data = [embeddings, texts]
    ids = collection.insert(data)
    collection.flush()
    return ids
ثم يعمل على استخراج النص من ملفات في مجلد ، وقم بتقطيع النص وترميزه. ملاحظة: إذا كانت لديك اقتراحات حول أفضل طريقة للتعامل مع طول القطعة وحجم البايت للمخطط الفعلي ، فقل:

def extract_text_from_pdf(pdf_path):
    # Implement PDF reading here
    # Example: Using PyPDF2 (pip install PyPDF2)
    reader = PdfReader(pdf_path)
    text = ''
    for page in reader.pages:
        text += page.extract_text()
    return text
def chunk_text(text, chunk_size=512):
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0

    for word in words:
        # Calculate the byte length if the word is added
        additional_length = len(word.encode('utf-8')) + (1 if current_chunk else 0)
        new_length = current_length + additional_length
        if additional_length <= chunk_size:
            current_chunk.append(word)
            current_length = new_length
        else:
            chunks.append(' '.join(current_chunk))
            current_chunk = [word]
            current_length = len(word.encode('utf-8'))

    # Add the last chunk if it exists
    if current_chunk:
        chunks.append(' '.join(current_chunk))

    return chunks
def encode_text(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    outputs = model(**inputs)
    return outputs.pooler_output.detach().numpy()[0] #assuming single embedding
def move_file_to_done(file_name, source_folder, destination_folder):
    # Construct full file paths
    source_path = os.path.join(source_folder, file_name)
    destination_path = os.path.join(destination_folder, file_name)

    # Move the file
    try:
        shutil.move(source_path, destination_path)
        print(f'File {file_name} moved to {destination_folder}')
    except FileNotFoundError:
        print(f'File {file_name} not found in {source_folder}')
    except Exception as e:
        print(f'Error moving file: {e}')
وأخيرا وظيفة لمعالجة الملفات في مجلد:

def process_files(in_directory, out_directory, collection):
    for filename in os.listdir(in_directory):
        if filename.endswith('.pdf'):
            print(f"Processing {filename}.")
            texts = []
            embeddings = []
            pdf_path = os.path.join(in_directory, filename)
            text = extract_text_from_pdf(pdf_path)
            text_chunks = chunk_text(text)

            for txt_chunk in text_chunks:
                embedding = encode_text(txt_chunk)
                texts.append(txt_chunk)
                embeddings.append(embedding)

            ids = insert_embeddings(collection, embeddings, texts)
            move_file_to_done(filename, in_directory, out_directory)
            print(f"Inserted {ids.insert_count} embeddings from {filename}.")

        elif filename.endswith('.txt'):
            print(f"Processing {filename}.")
            texts = []
            embeddings = []
            txt_path = os.path.join(in_directory, filename)
            text = extract_text_from_txt(txt_path)
            text_chunks = chunk_text(text)

            for txt_chunk in text_chunks:
                embedding = encode_text(txt_chunk)
                texts.append(txt_chunk)
                embeddings.append(embedding)

            ids = insert_embeddings(collection, embeddings, texts)
            move_file_to_done(filename, in_directory, out_directory)
            print(f"Inserted {ids.insert_count} embeddings from {filename}.")
تضمين ملف PDF واحد
أقوم بتوصيل وإنشاء مجموعة لتخزين عمليات التضمين الخاصة بي عن طريق تشغيل التعليمات البرمجية التالية:

connect_to_milvus()
collection_name = "my_test_rag"
dimension_size = 768
collection = create_collection(collection_name, dimension_size)
سيعيد هذا:

Connected to Milvus.
Collection my_test_rag does not exist.
Collection my_test_rag created.
ثم أقوم بتشغيل معالجة الملفات داخل دليل معين:

my_source_directory = './PDFs'
my_dest_directory = './donePDFs'

process_files(my_source_directory, my_dest_directory, collection)
تم تشغيل ملف PDF واحد ، تم إدراج 1523 تضمنا.

البحث في قاعدة بيانات المتجهات
يمكنني الآن الاستعلام عن قاعدة بيانات المتجهات واسترداد أجزاء النص ذات الصلة:

from pymilvus import connections, Collection, utility
from transformers import DPRQuestionEncoderTokenizer, DPRQuestionEncoder

tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
model = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")


collection = Collection(name="my_test_rag")
# Check if the index already exists, and create if it does not
if not collection.has_index():
    index_params = {
        "index_type": "IVF_FLAT",
        "params": {"nlist": 100},
        "metric_type": "L2"
    }
    collection.create_index(field_name="embeddings", index_params=index_params)

# Load the collection into memory
collection.load()
# Prepare a query text and encode it into a vector
query_text = "Flashblade S3 statistics"
inputs = tokenizer(query_text, return_tensors="pt")
query_embedding = model(**inputs).pooler_output.detach().numpy()

# Convert the embedding to a list of floats
query_embedding = query_embedding.tolist()

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=query_embedding,
    anns_field="embeddings",
    param=search_params,
    limit=5,
    expr=None,
    output_fields=["text"]
)
# Print the search results
for result in results:
    for hit in result:
        print(f"ID: {hit.id}, Score: {hit.distance}, Text: {hit.entity.get('text')}")
سيتم استرداد أجزاء النص المطابقة، على سبيل المثال:

ID: 450165976472778458, Score: 95.83973693847656,
Text: Hover over any part of a chart to display values for a specific point
in time. The values that appear in the point-in-time tooltips are rounded to
 two decimal places. The Performance Chart The following example displays
the IOPS statistics on the array at precisely 5:00:00 on March 12 . The
Performance panel includes the Latency, IOPS, and Bandwidth charts. Latency
 The Latency chart displays the average latency times for various operations.
 •Read Latency (R)
...
تضمين آلاف المستندات
إذن ماذا يحدث عندما أقوم بإنشاء مجموعة تستند إلى عدة جيجابايت من الملفات النصية؟

لهذا قمت بتنزيل 19101 ملف نصي من مشروع Gutenberg بحجم أولي إجمالي يبلغ 7.1 جيجابايت. باستخدام الكود أعلاه ، تم استيعاب مجموعة البيانات هذه في قاعدة بيانات المتجهات مما أدى إلى ما يزيد قليلا عن 14 مليون تضمين. يتم الحصول على العدد النهائي للكيانات في المجموعة باستخدام:

from pymilvus import connections, Collection, utility
# Connect to Milvus
connections.connect("default", host="1.2.3.4", port="19530")

collection_name = "my_test_rag"
collection = Collection(name=collection_name)
# Get the number of entities in the collection
num_entities = collection.num_entities
print(f"The number of entities in the collection '{collection_name}' is: {num_entities}")
نتيجة:

The number of entities in the collection 'my_test_rag' is: 14379441
الآن يقوم Milvus بإنشاء كائنات index_files و insert_log و stats_log. انتظرت تشغيل كل insert_logs في index_files وتشغيل الضغط و gc العميق على منصة Milvus للحصول على حجم التخزين النهائي لقاعدة البيانات. للحصول على إجمالي مساحة التخزين المستخدمة لمجموعتي ، أقوم بإدراج وتلخيص الكائنات التي تم إنشاؤها على طبقة التخزين S3:

aws s3 --endpoint-url=http://192.168.40.165 --profile fbstaines03 ls --summarize --human-readable --recursive s3://jbt-milvus/milvus/jbt-milvus/index_files | grep Total
Total Objects: 4362
   Total Size: 61.0 GiB

aws s3 --endpoint-url=http://192.168.40.165 --profile fbstaines03 ls --summarize --human-readable --recursive s3://jbt-milvus/milvus/jbt-milvus/insert_log | grep Total
Total Objects: 4030
   Total Size: 45.2 GiB
لقد شهدنا زيادة قدرها 8 أضعاف (أو 15 ضعفا إذا أخذنا في الاعتبار جميع ملفات سجل ديسيبل) في التخزين المطلوبة من مجموعة بيانات النص الأولي إلى إصدار قاعدة البيانات المتجه والمجزأ.

جانب آخر ملحوظ هو حقيقة أنه أثناء الاستيعاب ، ينمو حجم قاعدة البيانات إلى عدة أضعاف النتيجة النهائية! ويرجع ذلك إلى استيعاب البيانات في insert_log () ثم يتم تشغيلها في index_logs مع الضغط والضغط وجمع القمامة مما يساعد على تقليل مساحة التخزين التي تتطلبها قاعدة البيانات. خلال الاختبار أعلاه ، نمت قاعدة بياناتي إلى عدة مئات من الجيبي بايت ، أي عدة أضعاف الحجم الإجمالي الفعلي للنهاية. وفقا لقطة الشاشة التالية حيث أخذت قياسات لتطور استخدام تخزين S3 الخلفي على مدار عدة ساعات:


مثال على تذبذب مساحة تخزين Milvus S3 المستخدمة
استنتاج
يتضمن عبء عمل قاعدة بيانات متجهات المؤسسة ما يلي:

زيادة مساحة التخزين من البيانات الأولية إلى المتجهات في قاعدة البيانات
الحجم المتقلب أثناء استيعاب قاعدة بيانات vector
الاستيعاب المتكرر للحفاظ على معرفة المجال المحدثة القابلة للبحث
تعد منصة التخزين المبنية على البساطة والأداء المتسق وقابلية التوسع أمرا أساسيا عند التفكير في حالة استخدام RAG للمؤسسات ، فإننا نميل إلى نسيان أنه ليس كل شيء يتعلق بوحدة معالجة الرسومات!

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

للبحث عن النصوص الأكثر صلة باستعلام معين:

In [16]:
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text


In [17]:
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()


tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

In [18]:
import faiss
import numpy as np

# إعداد الفهرس
embedding_dim = 384  # حجم التضمين من النموذج
index = faiss.IndexFlatL2(embedding_dim)  # L2 هو معيار المسافة

# قائمة التضمينات والنصوص المرتبطة بها
embeddings = []
texts = []

# مثال على إضافة نصوص وتضميناتها إلى الفهرس
def add_to_faiss(text, index):
    embedding = generate_embedding(text)
    index.add(np.array([embedding], dtype="float32"))
    embeddings.append(embedding)
    texts.append(text)

# إضافة النصوص
add_to_faiss("هذا مثال نصي.", index)
add_to_faiss("نص آخر للاختبار.", index)


In [19]:
def search_in_faiss(query, index, k=5):
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# اختبار البحث
query = "اختبار النصوص"
results = search_in_faiss(query, index)
for result, distance in results:
    print(f"Result: {result}, Distance: {distance}")


Result: نص آخر للاختبار., Distance: 3.311124801635742
Result: هذا مثال نصي., Distance: 11.12571907043457
Result: نص آخر للاختبار., Distance: 3.4028234663852886e+38
Result: نص آخر للاختبار., Distance: 3.4028234663852886e+38
Result: نص آخر للاختبار., Distance: 3.4028234663852886e+38


%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

### العثور على النصوص المشابهة
### مفيد للتعامل مع ملف الداتا

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

شغال بحث وتطليع نتائج  مكشابهه من النص

نتائج عينة:
عند تشغيل الكود وإدخال استعلام مثل "Who is Percy Jackson?"، ستعرض النتائج نصوصًا من الكتاب متعلقة بالاستعلام.

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Result: this ever happened. But if you recognize yourself in these pages—if you feel something stirring inside—stop reading immediately. You might be one of us. And once you know that, it's only a mat­ ter of time before they sense it too, and they'll come for you. Don't say I didn't warn you. My name is Percy Jackson. I'm twelve years old. Until a few months ago, I was a boarding student at Yancy Academy, a private school for troubled kids in upstate New York. Am I a troubled kid? Yeah. You could say that.
Distance: 48.559165954589844

Result: Nancy hissed, her face even brighter red than her hair. At least Nancy got packed, too. Mr. Brunner was the only one who ever caught her saying anything wrong. He had radar ears. I thought about his question, and shrugged. "I don't know, sir." "I see." Mr. Brunner looked disappointed. "Well, half credit, Mr. Jackson. Zeus did indeed feed Kronos a mixture of mustard and wine, which made him disgorge his other five children, who, of course, being immortal gods, had been living and growing up completely
Distance: 51.04908752441406

Result: eating his kids, right?" "Yes," Mr. Brunner said, obviously not satisfied. "And he did this because . . ." "Well..." I racked my brain to remember. "Kronos was the king god, and—" "God?" Mr. Brunner asked. "Titan," I corrected myself. "And ... he didn't trust his kids, who were the gods. So, um, Kronos ate them, right? But his wife hid baby Zeus, and gave Kronos a rock to eat instead. And later, when Zeus grew up, he tricked his dad, Kronos, into barfing up his brothers and sisters—" "Eeew!" said
Distance: 51.074798583984375

Result: snickered something about the naked guy on the stele, and I turned around and said, "Will you shut up!" It came out louder than I meant it to. The whole group laughed. Mr. Brunner stopped his story. "Mr. Jackson," he said, "did you have a comment?" My face was totally red. I said, "No, sir." Mr. Brunner pointed to one of the pictures on the stele. "Perhaps you'll tell us what this picture represents?" I looked at the carving, and felt a flush of relief, because I actually recognized it. "That's Kronos
Distance: 51.2169189453125

Result: one of the girls behind me. "—and so there was this big fight between the gods and the Titans," I continued, "and the gods won." [5] Some snickers from the group. Behind me, Nancy Bobofit mumbled to a friend, "Like we're going to use this in real life. Like it's going to say on our job applications, 'Please explain why Kronos ate his kids.' " "And why, Mr. Jackson," Brunner said, "to paraphrase Miss Bobofit's excellent question, does this matter in real life?" "Busted," Grover muttered. "Shut up,"
Distance: 51.47588348388672

In [1]:
# تثبيت المكتبات الضرورية
#!pip install faiss-cpu PyPDF2 transformers

# استخراج النصوص من الكتاب باستخدام PyPDF2
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# تحديد مسار الكتاب
book_path = "/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"
book_text = extract_text_from_pdf(book_path)

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(text, max_chunk_size=500):
    words = text.split()
    chunks = []
    chunk = []
    for word in words:
        chunk.append(word)
        if len(" ".join(chunk)) > max_chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

text_chunks = split_text_into_chunks(book_text)

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

# إعداد الفهرس
embedding_dim = 384  # حجم التضمين من النموذج
index = faiss.IndexFlatL2(embedding_dim)  # L2 هو معيار المسافة

# إضافة النصوص والتضمينات إلى الفهرس
embeddings = []
texts = []

for chunk in text_chunks:
    embedding = generate_embedding(chunk)
    index.add(np.array([embedding], dtype="float32"))
    embeddings.append(embedding)
    texts.append(chunk)

# البحث في الكتاب باستخدام Faiss
def search_in_faiss(query, index, k=5):
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# اختبار البحث
query = "Who is Percy Jackson?"
results = search_in_faiss(query, index)

# عرض النتائج
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Result: this ever happened. But if you recognize yourself in these pages—if you feel something stirring inside—stop reading immediately. You might be one of us. And once you know that, it's only a mat­ ter of time before they sense it too, and they'll come for you. Don't say I didn't warn you. My name is Percy Jackson. I'm twelve years old. Until a few months ago, I was a boarding student at Yancy Academy, a private school for troubled kids in upstate New York. Am I a troubled kid? Yeah. You could say that.
Distance: 48.559165954589844

Result: Nancy hissed, her face even brighter red than her hair. At least Nancy got packed, too. Mr. Brunner was the only one who ever caught her saying anything wrong. He had radar ears. I thought about his question, and shrugged. "I don't know, sir." "I see." Mr. Brunner looked disappointed. "Well, half credit, Mr. Jackson. Zeus did indeed feed Kronos a mixture of mustard and wine, which made him disgorge his other five children, who, of course, being 

كود تنظيف النصوص

شغال

In [2]:
import re

def clean_text(text):
    """
    وظيفة لتنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    # إزالة الحواشي مثل [1], [5], [A], إلخ.
    text = re.sub(r'\[\d+\]', '', text)

    # إزالة العلامات الخاصة (يمكن تعديلها حسب الحاجة)
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)

    # إزالة الأسطر الفارغة أو الزائدة
    text = re.sub(r'\n\s*\n', '\n', text)

    # إزالة المسافات الزائدة في بداية ونهاية النص
    text = text.strip()

    return text

# مثال على النصوص:
sample_text = """
[1] My name is Percy Jackson. I'm twelve years old.
Until a few months ago, I was at Yancy Academy. [5]

---

Am I a troubled kid? Yeah. You could say that.
"""

# تنظيف النصوص
cleaned_text = clean_text(sample_text)
print(cleaned_text)


My name is Percy Jackson. Im twelve years old. 
Until a few months ago, I was at Yancy Academy. 
Am I a troubled kid? Yeah. You could say that.


############################################

### شغال

الكود الكامل مع تنظيف النصوص

In [1]:
# تثبيت المكتبات الضرورية
#!pip install faiss-cpu PyPDF2 transformers

# استخراج النصوص من الكتاب باستخدام PyPDF2
from PyPDF2 import PdfReader

def extract_text_from_pdf(file_path):
    reader = PdfReader(file_path)
    text = ""
    for page in reader.pages:
        text += page.extract_text()
    return text

# تنظيف النصوص
import re

def clean_text(text):
    """
    وظيفة لتنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    text = re.sub(r'\[\d+\]', '', text)  # إزالة الحواشي
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)  # إزالة العلامات الخاصة
    text = re.sub(r'\n\s*\n', '\n', text)  # إزالة الأسطر الفارغة
    text = text.strip()  # إزالة المسافات الزائدة
    return text

# تحديد مسار الكتاب
book_path = "/content/The_Lightning_Thief_-_Percy_Jackson_1-10.pdf"
book_text = extract_text_from_pdf(book_path)

# تنظيف النص المستخرج
cleaned_book_text = clean_text(book_text)

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(text, max_chunk_size=500):
    words = text.split()
    chunks = []
    chunk = []
    for word in words:
        chunk.append(word)
        if len(" ".join(chunk)) > max_chunk_size:
            chunks.append(" ".join(chunk))
            chunk = []
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

text_chunks = split_text_into_chunks(cleaned_book_text)

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

# إعداد الفهرس
embedding_dim = 384  # حجم التضمين من النموذج
index = faiss.IndexFlatL2(embedding_dim)  # L2 هو معيار المسافة

# إضافة النصوص والتضمينات إلى الفهرس
embeddings = []
texts = []

for chunk in text_chunks:
    embedding = generate_embedding(chunk)
    index.add(np.array([embedding], dtype="float32"))
    embeddings.append(embedding)
    texts.append(chunk)

# البحث في الكتاب باستخدام Faiss
def search_in_faiss(query, index, k=5):
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# اختبار البحث
query = "Who is Percy Jackson?"
results = search_in_faiss(query, index)

# عرض النتائج
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Result: pagesif you feel something stirring insidestop reading immediately. You might be one of us. And once you know that, its only a mat ter of time before they sense it too, and theyll come for you. Dont say I didnt warn you. My name is Percy Jackson. Im twelve years old. Until a few months ago, I was a boarding student at Yancy Academy, a private school for troubled kids in upstate New York. Am I a troubled kid? Yeah. You could say that. I could start at any point in my short miserable life to prove it,
Distance: 46.348480224609375

Result: his question, and shrugged. I dont know, sir. I see. Mr. Brunner looked disappointed. Well, half credit, Mr. Jackson. Zeus did indeed feed Kronos a mixture of mustard and wine, which made him disgorge his other five children, who, of course, being immortal gods, had been living and growing up completely undigested in the Titans stomach. The gods defeated their father, sliced him to pieces with his own scythe, and scattered his remains in Tartaru

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning:
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Result: pagesif you feel something stirring insidestop reading immediately. You might be one of us. And once you know that, its only a mat ter of time before they sense it too, and theyll come for you. Dont say I didnt warn you. My name is Percy Jackson. Im twelve years old. Until a few months ago, I was a boarding student at Yancy Academy, a private school for troubled kids in upstate New York. Am I a troubled kid? Yeah. You could say that. I could start at any point in my short miserable life to prove it,
Distance: 46.348480224609375

Result: his question, and shrugged. I dont know, sir. I see. Mr. Brunner looked disappointed. Well, half credit, Mr. Jackson. Zeus did indeed feed Kronos a mixture of mustard and wine, which made him disgorge his other five children, who, of course, being immortal gods, had been living and growing up completely undigested in the Titans stomach. The gods defeated their father, sliced him to pieces with his own scythe, and scattered his remains in Tartarus, the darkest part of the Underworld. On that happy
Distance: 48.22768783569336

Result: but things really started going bad last May, when our sixthgrade class took a field trip to Manhattan twentyeight mentalcase kids and two teachers on a yellow school bus, heading to the Metropolitan Museum of Art to look at ancient Greek and Roman stuff. I knowit sounds like torture. Most Yancy field trips were. But Mr. Brunner, our Latin teacher, was leading this trip, so I had hopes. Mr. Brunner was this middleaged guy in a motorized wheelchair. He had thinning hair and a scruffy beard and a frayed
Distance: 51.83222961425781

Result: was the king god, and God? Mr. Brunner asked. Titan, I corrected myself. And ... he didnt trust his kids, who were the gods. So, um, Kronos ate them, right? But his wife hid baby Zeus, and gave Kronos a rock to eat instead. And later, when Zeus grew up, he tricked his dad, Kronos, into barfing up his brothers and sisters Eeew! said one of the girls behind me. and so there was this big fight between the gods and the Titans, I continued, and the gods won. Some snickers from the group. Behind me, Nancy
Distance: 52.132896423339844

Result: it to. The whole group laughed. Mr. Brunner stopped his story. Mr. Jackson, he said, did you have a comment? My face was totally red. I said, No, sir. Mr. Brunner pointed to one of the pictures on the stele. Perhaps youll tell us what this picture represents? I looked at the carving, and felt a flush of relief, because I actually recognized it. Thats Kronos eating his kids, right? Yes, Mr. Brunner said, obviously not satisfied. And he did this because . . . Well... I racked my brain to remember. Kronos
Distance: 52.8665771484375


$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@

In [ ]:
Inventor

In [1]:
# تثبيت المكتبات الضرورية
#!pip install faiss-cpu pandas transformers

# استخراج النصوص من ملف CSV باستخدام pandas
import pandas as pd

def extract_text_from_csv(file_path, Inventor):
    """
    وظيفة لاستخراج النصوص من عمود معين في ملف CSV.
    - file_path: مسار ملف CSV.
    - text_column: اسم العمود الذي يحتوي على النصوص.
    """
    df = pd.read_csv(file_path)
    if Inventor not in df.columns:
        raise ValueError(f"العمود '{Inventor}' غير موجود في ملف CSV.")
    return df[Inventor].dropna().tolist()  # استخراج النصوص وتجاهل القيم الفارغة

# تنظيف النصوص
import re

def clean_text(text):
    """
    وظيفة لتنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    text = re.sub(r'\[\d+\]', '', text)  # إزالة الحواشي
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)  # إزالة العلامات الخاصة
    text = re.sub(r'\n\s*\n', '\n', text)  # إزالة الأسطر الفارغة
    text = text.strip()  # إزالة المسافات الزائدة
    return text

# تحديد مسار ملف CSV واسم العمود
csv_path = "/content/inventors_and_inventions.csv"  # ضع مسار ملف CSV هنا
text_column = "text"  # اسم العمود الذي يحتوي على النصوص
raw_texts = extract_text_from_csv(csv_path, text_column)

# تنظيف النصوص المستخرجة
cleaned_texts = [clean_text(text) for text in raw_texts]

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(texts, max_chunk_size=500):
    chunks = []
    for text in texts:
        words = text.split()
        chunk = []
        for word in words:
            chunk.append(word)
            if len(" ".join(chunk)) > max_chunk_size:
                chunks.append(" ".join(chunk))
                chunk = []
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

text_chunks = split_text_into_chunks(cleaned_texts)

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

# إعداد الفهرس
embedding_dim = 384  # حجم التضمين من النموذج
index = faiss.IndexFlatL2(embedding_dim)  # L2 هو معيار المسافة

# إضافة النصوص والتضمينات إلى الفهرس
embeddings = []
texts = []

for chunk in text_chunks:
    embedding = generate_embedding(chunk)
    index.add(np.array([embedding], dtype="float32"))
    embeddings.append(embedding)
    texts.append(chunk)

# البحث في النصوص باستخدام Faiss
def search_in_faiss(query, index, k=5):
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# اختبار البحث
query = "What did Thomas Edison invent?"
results = search_in_faiss(query, index)

# عرض النتائج
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


ValueError: العمود 'text' غير موجود في ملف CSV.

In [2]:
# تثبيت المكتبات الضرورية
#!pip install faiss-cpu pandas transformers

# استخراج النصوص من ملف CSV باستخدام pandas
import pandas as pd

def extract_text_from_csv(file_path, text_column):
    """
    وظيفة لاستخراج النصوص من عمود معين في ملف CSV.
    - file_path: مسار ملف CSV.
    - text_column: اسم العمود الذي يحتوي على النصوص.
    """
    df = pd.read_csv(file_path)
    if text_column not in df.columns:
        raise ValueError(f"العمود '{text_column}' غير موجود في ملف CSV.")
    return df[text_column].dropna().tolist()  # استخراج النصوص وتجاهل القيم الفارغة

# تنظيف النصوص
import re

def clean_text(text):
    """
    وظيفة لتنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    text = re.sub(r'\[\d+\]', '', text)  # إزالة الحواشي
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)  # إزالة العلامات الخاصة
    text = re.sub(r'\n\s*\n', '\n', text)  # إزالة الأسطر الفارغة
    text = text.strip()  # إزالة المسافات الزائدة
    return text

# تحديد مسار ملف CSV واسم العمود
csv_path = "/content/inventors_and_inventions.csv"  # ضع مسار ملف CSV هنا
text_column = "text"  # اسم العمود الذي يحتوي على النصوص
raw_texts = extract_text_from_csv(csv_path, text_column)

# تنظيف النصوص المستخرجة
cleaned_texts = [clean_text(text) for text in raw_texts]

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(texts, max_chunk_size=500):
    chunks = []
    for text in texts:
        words = text.split()
        chunk = []
        for word in words:
            chunk.append(word)
            if len(" ".join(chunk)) > max_chunk_size:
                chunks.append(" ".join(chunk))
                chunk = []
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

text_chunks = split_text_into_chunks(cleaned_texts)

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

# إعداد الفهرس
embedding_dim = 384  # حجم التضمين من النموذج
index = faiss.IndexFlatL2(embedding_dim)  # L2 هو معيار المسافة

# إضافة النصوص والتضمينات إلى الفهرس
embeddings = []
texts = []

for chunk in text_chunks:
    embedding = generate_embedding(chunk)
    index.add(np.array([embedding], dtype="float32"))
    embeddings.append(embedding)
    texts.append(chunk)

# البحث في النصوص باستخدام Faiss
def search_in_faiss(query, index, k=5):
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# اختبار البحث
query = "What did Thomas Edison invent?"
results = search_in_faiss(query, index)

# عرض النتائج
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


ValueError: العمود 'text' غير موجود في ملف CSV.

import pandas as pd

csv_path = "/content/dataset.csv"  # ضع مسار ملف CSV هنا
df = pd.read_csv(csv_path)

# عرض أسماء الأعمدة
print(df.columns)


ماذا تفعل إذا كنت غير متأكد من اسم العمود؟
يمكنك عرض أسماء الأعمدة في ملف CSV باستخدام الكود التالي:

شغال

In [3]:
import pandas as pd

csv_path = "/content/inventors_and_inventions.csv"  # ضع مسار ملف CSV هنا
df = pd.read_csv(csv_path)

# عرض أسماء الأعمدة
print(df.columns)


Index(['Inventor', 'Invention'], dtype='object')


In [ ]:
# تحديد مسار ملف CSV واسم العمود
csv_path = "/content/dataset.csv"  # ضع مسار ملف CSV هنا
text_column = "Inventor"  # اسم العمود الذي يحتوي على النصوص
raw_texts = extract_text_from_csv(csv_path, text_column)

# تنظيف النصوص المستخرجة
cleaned_texts = [clean_text(text) for text in raw_texts]

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
text_chunks = split_text_into_chunks(cleaned_texts)

# باقي الكود كما هو# تثبيت المكتبات الضرورية
#!pip install faiss-cpu pandas transformers

# استخراج النصوص من ملف CSV باستخدام pandas
import pandas as pd

def extract_text_from_csv(file_path, Inventor):
    """
    وظيفة لاستخراج النصوص من عمود معين في ملف CSV.
    - file_path: مسار ملف CSV.
    - text_column: اسم العمود الذي يحتوي على النصوص.
    """
    df = pd.read_csv(file_path)
    if Inventor not in df.columns:
        raise ValueError(f"العمود '{Inventor}' غير موجود في ملف CSV.")
    return df[Inventor].dropna().tolist()  # استخراج النصوص وتجاهل القيم الفارغة

# تنظيف النصوص
import re

def clean_text(text):
    """
    وظيفة لتنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    text = re.sub(r'\[\d+\]', '', text)  # إزالة الحواشي
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)  # إزالة العلامات الخاصة
    text = re.sub(r'\n\s*\n', '\n', text)  # إزالة الأسطر الفارغة
    text = text.strip()  # إزالة المسافات الزائدة
    return text

# تحديد مسار ملف CSV واسم العمود
csv_path = "/content/inventors_and_inventions.csv"  # ضع مسار ملف CSV هنا
text_column = "text"  # اسم العمود الذي يحتوي على النصوص
raw_texts = extract_text_from_csv(csv_path, text_column)

# تنظيف النصوص المستخرجة
cleaned_texts = [clean_text(text) for text in raw_texts]

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(texts, max_chunk_size=500):
    chunks = []
    for text in texts:
        words = text.split()
        chunk = []
        for word in words:
            chunk.append(word)
            if len(" ".join(chunk)) > max_chunk_size:
                chunks.append(" ".join(chunk))
                chunk = []
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

text_chunks = split_text_into_chunks(cleaned_texts)

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

# إعداد الفهرس
embedding_dim = 384  # حجم التضمين من النموذج
index = faiss.IndexFlatL2(embedding_dim)  # L2 هو معيار المسافة

# إضافة النصوص والتضمينات إلى الفهرس
embeddings = []
texts = []

for chunk in text_chunks:
    embedding = generate_embedding(chunk)
    index.add(np.array([embedding], dtype="float32"))
    embeddings.append(embedding)
    texts.append(chunk)

# البحث في النصوص باستخدام Faiss
def search_in_faiss(query, index, k=5):
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# اختبار البحث
query = "What did Thomas Edison invent?"
results = search_in_faiss(query, index)

# عرض النتائج
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


شغال مع داتا csvيخرج النص المشابه للسؤال

In [4]:
# تثبيت المكتبات الضرورية
# !pip install faiss-cpu transformers pandas

# استخراج النصوص من ملف CSV
import pandas as pd

def extract_text_from_csv(file_path, column_name):
    """
    استخراج النصوص من عمود محدد في ملف CSV.
    """
    try:
        df = pd.read_csv(file_path)
        if column_name not in df.columns:
            raise ValueError(f"العمود '{column_name}' غير موجود في ملف CSV.")
        return df[column_name].dropna().tolist()
    except Exception as e:
        raise ValueError(f"حدث خطأ أثناء قراءة ملف CSV: {e}")

# تنظيف النصوص
import re

def clean_text(text):
    """
    تنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    text = re.sub(r'\[\d+\]', '', text)  # إزالة الحواشي
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)  # إزالة العلامات الخاصة
    text = re.sub(r'\n\s*\n', '\n', text)  # إزالة الأسطر الفارغة
    text = text.strip()  # إزالة المسافات الزائدة
    return text

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(text_list, max_chunk_size=500):
    chunks = []
    for text in text_list:
        words = text.split()
        chunk = []
        for word in words:
            chunk.append(word)
            if len(" ".join(chunk)) > max_chunk_size:
                chunks.append(" ".join(chunk))
                chunk = []
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    """
    إنشاء تضمين للنص باستخدام نموذج Transformers.
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

def build_faiss_index(embeddings, dimension=384):
    """
    إعداد فهرس Faiss وإضافة التضمينات.
    """
    index = faiss.IndexFlatL2(dimension)  # L2 هو معيار المسافة
    index.add(np.array(embeddings, dtype="float32"))
    return index

# البحث في النصوص باستخدام Faiss
def search_in_faiss(query, index, texts, k=5):
    """
    البحث في النصوص باستخدام Faiss.
    """
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# مسار ملف CSV واسم العمود
csv_path = "/content/inventors_and_inventions.csv"  # ضع مسار ملف CSV هنا
text_column = "Inventor"  # اسم العمود الذي يحتوي على النصوص

# استخراج النصوص وتنظيفها
raw_texts = extract_text_from_csv(csv_path, text_column)
cleaned_texts = [clean_text(text) for text in raw_texts]

# تقسيم النصوص إلى فقرات قصيرة
text_chunks = split_text_into_chunks(cleaned_texts)

# إنشاء التضمينات وبناء الفهرس
embeddings = [generate_embedding(chunk) for chunk in text_chunks]
index = build_faiss_index(embeddings)

# اختبار البحث
query = "Innovations in technology"  # استعلام البحث
results = search_in_faiss(query, index, text_chunks)

# عرض النتائج
print("نتائج البحث:")
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


نتائج البحث:
Result: Galileo Galilei
Distance: 66.47154998779297

Result: Steve Jobs
Distance: 72.65122985839844

Result: Guglielmo Marconi
Distance: 72.67211151123047

Result: Bill Gates
Distance: 75.57966613769531

Result: Thomas Edison
Distance: 78.95977020263672



على سبيل المثال:

إذا أردت إرجاع 10 نتائج بدلاً من 5، يمكنك تعديل الكود كالتالي:
python
Copy
Edit
results = search_in_faiss(query, index, text_chunks, k=10)
بذلك ستتمكن من الحصول على عدد أكبر من النتائج.

In [6]:
# تثبيت المكتبات الضرورية
# !pip install faiss-cpu transformers pandas

# استخراج النصوص من ملف CSV
import pandas as pd

def extract_text_from_csv(file_path, column_name):
    """
    استخراج النصوص من عمود محدد في ملف CSV.
    """
    try:
        df = pd.read_csv(file_path)
        if column_name not in df.columns:
            raise ValueError(f"العمود '{column_name}' غير موجود في ملف CSV.")
        return df[column_name].dropna().tolist()
    except Exception as e:
        raise ValueError(f"حدث خطأ أثناء قراءة ملف CSV: {e}")

# تنظيف النصوص
import re

def clean_text(text):
    """
    تنظيف النصوص:
    1. إزالة الحواشي (مثل [1] أو [5]).
    2. إزالة الأسطر الفارغة أو المسافات الزائدة.
    3. إزالة العلامات غير المرغوب فيها (مثل !، #، "، إلخ).
    """
    text = re.sub(r'\[\d+\]', '', text)  # إزالة الحواشي
    text = re.sub(r'[^\w\s.,?!:؛،]', '', text)  # إزالة العلامات الخاصة
    text = re.sub(r'\n\s*\n', '\n', text)  # إزالة الأسطر الفارغة
    text = text.strip()  # إزالة المسافات الزائدة
    return text

# تقسيم النصوص إلى فقرات قصيرة لتسهيل التضمين
def split_text_into_chunks(text_list, max_chunk_size=500):
    chunks = []
    for text in text_list:
        words = text.split()
        chunk = []
        for word in words:
            chunk.append(word)
            if len(" ".join(chunk)) > max_chunk_size:
                chunks.append(" ".join(chunk))
                chunk = []
        if chunk:
            chunks.append(" ".join(chunk))
    return chunks

# إنشاء التضمينات باستخدام Transformers
from transformers import AutoTokenizer, AutoModel
import torch

# تحميل النموذج
tokenizer = AutoTokenizer.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")
model = AutoModel.from_pretrained("sentence-transformers/all-MiniLM-L6-v2")

def generate_embedding(text):
    """
    إنشاء تضمين للنص باستخدام نموذج Transformers.
    """
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    # استخدام متوسط التضمينات
    embeddings = outputs.last_hidden_state.mean(dim=1)
    return embeddings.squeeze().numpy()

# إعداد Faiss وإضافة التضمينات
import faiss
import numpy as np

def build_faiss_index(embeddings, dimension=384):
    """
    إعداد فهرس Faiss وإضافة التضمينات.
    """
    index = faiss.IndexFlatL2(dimension)  # L2 هو معيار المسافة
    index.add(np.array(embeddings, dtype="float32"))
    return index

# البحث في النصوص باستخدام Faiss
def search_in_faiss(query, index, texts, k=1):
    """
    البحث في النصوص باستخدام Faiss.
    """
    query_embedding = generate_embedding(query)
    distances, indices = index.search(np.array([query_embedding], dtype="float32"), k)
    results = [(texts[i], distances[0][j]) for j, i in enumerate(indices[0])]
    return results

# مسار ملف CSV واسم العمود
csv_path = "/content/inventors_and_inventions.csv"  # ضع مسار ملف CSV هنا
text_column = "Inventor"  # اسم العمود الذي يحتوي على النصوص

# استخراج النصوص وتنظيفها
raw_texts = extract_text_from_csv(csv_path, text_column)
cleaned_texts = [clean_text(text) for text in raw_texts]

# تقسيم النصوص إلى فقرات قصيرة
text_chunks = split_text_into_chunks(cleaned_texts)

# إنشاء التضمينات وبناء الفهرس
embeddings = [generate_embedding(chunk) for chunk in text_chunks]
index = build_faiss_index(embeddings)

# اختبار البحث
query = "Thomas Edison"  # استعلام البحث
results = search_in_faiss(query, index, text_chunks)

# عرض النتائج
print("نتائج البحث:")
for result, distance in results:
    print(f"Result: {result}\nDistance: {distance}\n")


نتائج البحث:
Result: Thomas Edison
Distance: 0.0

